In [16]:
from model import GeneratorModule
from scripts.datasetscripts import parse_concatenated_json, format_augmented_goal
import torch

In [2]:
ckpt_path = "/Users/artem.semidetnov/Desktop/MetaGenerator_v1/.cadence/cache/id2f7519f525f7457f899938c06c46d14c/45208/outputs/lightning_logs/version_5/checkpoints/last.ckpt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GeneratorModule.load_from_checkpoint(ckpt_path, strict=False)
model.to(device)
model.eval()

GeneratorModule(
  (generator): T5ForConditionalGeneration(
    (shared): Embedding(384, 1472)
    (encoder): T5Stack(
      (embed_tokens): Embedding(384, 1472)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1472, out_features=384, bias=False)
                (k): Linear(in_features=1472, out_features=384, bias=False)
                (v): Linear(in_features=1472, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=1472, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=1472, out_features=3584, bias=False)
                (wi_1)

In [19]:
test_prompts = parse_concatenated_json('/Users/artem.semidetnov/Desktop/MetaGenerator_v1/dataset_small/test.json')
def test_model(prompt_index : int) -> None:
    prompt = test_prompts[prompt_index]

    prompt_ = format_augmented_goal(
            prompt["Expected type"],
            prompt["Premises"],
            512,
            0.0,
        )
    prompts = [prompt_]
    enc = model.tokenizer(
        prompts,
        padding=True,
        truncation=True,
        max_length=getattr(model.hparams, "max_inp_seq_len", 512),
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        outputs = model.generator.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"],
            num_beams=getattr(model.hparams, "num_beams", 4),
            length_penalty=getattr(model.hparams, "length_penalty", 0.0),
            max_length=getattr(model.hparams, "max_oup_seq_len", 128),
        )
    preds = model.tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for i, p in enumerate(preds):
        print(f"model output = {p} \n true value = {prompt['Expression']}")

In [36]:
test_model(11)

model output = prop-dpi (\lam i => ext_coe) (closure1_LDiv.1 (closure1_LDiv.1 $ inP (1, inP (1, inP (1, inP (1, inP (1, inP (1, inP (1, inP (1 
 true value = prop-dpi (\lam i => ext_coe) (contains_zro {KernelC {IntRing} {FinRing {n}} (intMap {FinRing {n}})})
  (contains_zro {closure1 {IntEuclidean} (pos (suc n))}) @ i


In [37]:
!wc -l /Users/artem.semidetnov/Desktop/MetaGenerator_v1/dataset_small/train.json

   10000 /Users/artem.semidetnov/Desktop/MetaGenerator_v1/dataset_small/train.json


[0] rewrite (get-leaf {this}) (rewrite (get-leaf {this}) (rewrite (get-leaf {this}) idp))


In [8]:
model.training

False